# With ChatModel

In [10]:
import json

from datetime import datetime

from langchain.chat_models import ChatOpenAI
from langchain.prompts import load_prompt
from langchain.schema import StrOutputParser

In [11]:
def save_result(output, prompt_type, model, init, best_practice):
    now = datetime.now()
    date = now.strftime("%Y-%m-%d")
    time = now.strftime("%H:%M:%S")

    result = {}
    result['model'] = model
    result['creation_timestamp'] = date + ' ' + time
    result['prompts'] = {
        'init_uri': init,
        'best_practice_uri': best_practice}
    result['output'] = output

    with open('results/'+prompt_type+'/output_'+date+'_'+time+'.json', 'w') as f:
        json.dump(result, f, indent=4)

In [12]:
model_name = 'gpt-4'

model = ChatOpenAI(
    model=model_name,
    api_key=open('api.txt', 'r').read(),
    temperature=0
)

## Summary Length

In [7]:
with open('./data/json/last_row.json') as f:
    json_sample = json.load(f)

In [8]:
init_prompt_name = 'initPrompt_V1.1.0.json'
init_prompt = load_prompt('prompts/init/'+init_prompt_name)

sumLen_prompt_name = 'summaryLengthPrompt_V1.0.2.json'
sumLen_prompt = load_prompt('prompts/summary/'+sumLen_prompt_name).format(min= 6, max= 10)

In [9]:
chain = init_prompt | model | StrOutputParser()
print(init_prompt.format(best_practice= sumLen_prompt, ticket=json_sample))

 ### Instruction ###
Act as a professional Software Engineer working with issue trackers like Jira or Azure DevOps.
Working with these issue trackers, you must consider multiple text-based best practices to increase understandability and reduce time.
Revise a ticket for compliance with these best practices and offer recommendations if needed.
The best practice to check is: 

Best Practice: Summary Length
Your Task is to check if the ticket summary is between 6 and 10 words.
If the word count is in this range, return the original summary.
Otherwise, recommend a new summary for the ticket.
Please return only the summary string and nothing else.


### Context ###
A ticket consists of multiple fields. You are provided with the fields and a short description for each field.
Assignee: The person responsible to resolve the issue.
Comments: Community discussion on the issue, including author, timestamp and content.
Components: Project components to which the issue belongs.
CreatedDate: The tim

In [10]:
output = chain.invoke({"best_practice": sumLen_prompt,"ticket": json_sample})
output

'"Sourcetree crashes upon opening"'

In [11]:
save_result(output, 'summary', model_name, init_prompt_name, sumLen_prompt_name)


In [9]:
# # maybe usefull for later
# def run(model_name, init_prompt_name, sumLen_prompt_name):
#     model = ChatOpenAI(
#         model=model_name,
#         api_key=open('api.txt', 'r').read(),
#         temperature=0
#     )
#     with open('./data/json/last_row.json') as f:
#         json_sample = json.load(f)

#     init_prompt = load_prompt('prompts/init/'+init_prompt_name)
#     sumLen_prompt = load_prompt('prompts/summary/'+sumLen_prompt_name).format(min= 6, max= 10)
    
#     chain = init_prompt | model | StrOutputParser()

#     output = chain.invoke({"best_practice": sumLen_prompt,"ticket": json_sample})

#     save_result(output, 'summary', model_name, init_prompt_name, sumLen_prompt_name)  

# run('gpt-4', 'initPrompt_V1.0.0.json', 'summaryLengthPrompt_V1.0.1.json')  

## Description Length

In [23]:
with open('./data/json/last_row.json') as f:
    json_sample = json.load(f)

In [28]:
init_prompt_name = 'initPrompt_V1.1.0.json'
init_prompt = load_prompt('prompts/init/'+init_prompt_name)

descLen_prompt_name = 'descriptionLengthPrompt_V1.0.0.json'
descLen_prompt = load_prompt('prompts/description/'+descLen_prompt_name).format(min= 29, max= 111)

chain = init_prompt | model | StrOutputParser()

In [29]:
output = chain.invoke({"best_practice": descLen_prompt, "ticket": json_sample})
output

"'Source tree crashes every time I open it. I have uninstalled and reinstalled the software multiple times but the issue persists. This problem is affecting my work as I am unable to access my projects. I am currently using version 3.4.4 of Sourcetree for Windows. I have not been able to find a solution to this problem online or in the Sourcetree community forums. I would appreciate any assistance or suggestions on how to resolve this issue.'"

In [30]:
save_result(output, 'description', model_name, init_prompt_name, descLen_prompt_name)

## Field Update

In [18]:
with open('./data/json/last_row.json') as f:
    json_sample = json.load(f)

In [19]:
init_prompt_name = 'initPrompt_V1.1.0.json'
init_prompt = load_prompt('prompts/init/'+init_prompt_name)

update_prompt_name = 'updatePrompt_V1.2.0.json'
update_prompt = load_prompt('prompts/update/'+update_prompt_name).format()

chain = init_prompt | model | StrOutputParser()

In [20]:
output = chain.invoke({"best_practice": update_prompt,"ticket": json_sample})
output

"Changes Made:\n1. Updated 'VersionsFixed' field from None to '3.4.5' based on the comment by Oleksandr Naumenko.\n2. Updated 'Assignee' field from None to 'Vipin Yadav' as he was actively involved in resolving the issue.\n3. Updated 'TimeSpent' field from None to '2 months' based on the 'CreatedDate' and 'ResolvedDate' fields.\n4. Added 'Software Update' to 'Labels' field as the issue was resolved by updating the software.\n\nRecommendations:\n1. Always update the 'Assignee' field to reflect who is responsible for the issue.\n2. Update the 'TimeSpent' field to reflect the time taken to resolve the issue.\n3. Update the 'VersionsFixed' field once the issue is resolved in a particular version.\n4. Use the 'Labels' field to categorize the issue for better tracking and understanding."

In [21]:
save_result(output, 'update', model_name, init_prompt_name, update_prompt_name)